In [ ]:
'''
Modification of DNN_model.
'''

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
from utils import setup_seed, metric_auc
from sklearn import preprocessing

setup_seed(2022)

In [ ]:
train_ = preprocessing.normalize(np.load("../data/datasets/train_tensor_csigma005.npy", allow_pickle=True))
valid_ = preprocessing.normalize(np.load("../data/datasets/val_tensor_csigma005.npy", allow_pickle=True))
test = preprocessing.normalize(np.load("../data/datasets/test_tensor_csigma005.npy", allow_pickle=True))
train_lab_ = np.load("../data/datasets/train_label_csima005.npy", allow_pickle=True)
valid_lab_ = np.load("../data/datasets/val_label_csima005.npy", allow_pickle=True)
test_lab = np.load("../data/datasets/test_label_csima005.npy", allow_pickle=True)

### Make Datasets and DataLoader

In [2]:
class MakeDatasets(Dataset):
    def __init__(self, data_path, label_path):
        self.tensor = np.load(data_path, allow_pickle = True)
        self.labels = np.load(label_path, allow_pickle = True)
        self.tensor = preprocessing.normalize(self.tensor)
        self.tensor = torch.tensor(self.tensor)
        self.labels = torch.tensor(self.labels, dtype = torch.float32).reshape((len(self.labels),1))
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        vector = self.tensor[idx,:]
        label = self.labels[idx]
        return vector, label

In [3]:
train_data = MakeDatasets(data_path = "../data/datasets/train_tensor_csigma005.npy", label_path = "../data/datasets/train_label_csima005.npy")
test_data = MakeDatasets(data_path = "../data/datasets/val_tensor_csigma005.npy", label_path = "../data/datasets/val_label_csima005.npy")

In [4]:
batch_size = 256
train_loader = DataLoader(train_data, batch_size = batch_size, shuffle = True)
test_loader = DataLoader(test_data, batch_size = batch_size, shuffle = True)

### Constructing Neural Networks

In [5]:
class Params():
    def __init__(self):
        self.dim1 = 841
        self.dim2 = 512
        self.dim3 = 256
        self.dim4 = 128
        self.dim5 = 64
        self.dim6 = 10
        self.lr = 0.001
        self.momentum = 0.9
        self.epochs = 10000
        self.dropout1 = 0.3
        self.dtopout2 = 0.2
params = Params()

In [6]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# Define model
class DNN(nn.Module):
    def __init__(self, params):
        super().__init__()
        self.linear1 = nn.Linear(params.dim1, params.dim2)
        self.bn1 = nn.BatchNorm1d(params.dim2)
        self.linear2 = nn.Linear(params.dim2, params.dim3)
        self.bn2 = nn.BatchNorm1d(params.dim3)
        self.linear3 = nn.Linear(params.dim3, params.dim5)
        self.bn3 = nn.BatchNorm1d(params.dim5)
        self.linear4 = nn.Linear(params.dim5, params.dim6)
        self.bn4 = nn.BatchNorm1d(params.dim6)
        self.linear5 = nn.Linear(params.dim6, params.dim1)
    def forward(self, x):
        x = self.linear1(x)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.linear2(x)
        x = self.bn2(x)
        x = F.relu(x)
        x = self.linear3(x)
        x = self.bn3(x)
        x = F.relu(x)
        x = self.linear4(x)
        x = self.bn4(x)
        x = F.relu(x)
        x = self.linear5(x)
        x = torch.sigmoid(x)
        return x

model = DNN(params).to(device)
print(model)

Using cpu device
DNN(
  (linear1): Linear(in_features=841, out_features=512, bias=True)
  (bn1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linear2): Linear(in_features=512, out_features=256, bias=True)
  (bn2): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linear3): Linear(in_features=256, out_features=64, bias=True)
  (bn3): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linear4): Linear(in_features=64, out_features=10, bias=True)
  (bn4): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linear5): Linear(in_features=10, out_features=841, bias=True)
)


### Train and evaluate

In [7]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    pred_arr = np.array([])
    label_arr = np.array([])
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        pred_arr = np.hstack((pred_arr, np.reshape(pred.detach().numpy(), len(pred))))
        label_arr = np.hstack((label_arr, np.reshape(np.array(y), len(y))))
        
        if batch %  200== 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
    auc = metric_auc(pred_arr, label_arr)
    print(f"Train AUC: {auc}")


def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        pred_arr = np.array([])
        label_arr = np.array([])
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            
            pred_arr = np.hstack((pred_arr, np.reshape(pred.detach().numpy(), len(pred))))
            label_arr = np.hstack((label_arr, np.reshape(np.array(y), len(y))))
    auc = metric_auc(pred_arr, label_arr)
    print(f"Validation AUC: {auc}")
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [8]:
loss_fn = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = params.lr, momentum = params.momentum)
#optimizer = torch.optim.Adam(model.parameters(), lr=0.005)

epochs = params.epochs
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_loader, model, loss_fn, optimizer)
    test_loop(test_loader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------


ValueError: Using a target size (torch.Size([256, 1])) that is different to the input size (torch.Size([256, 841])) is deprecated. Please ensure they have the same size.